![Logo](https://www.itu.int/web/pp-18/assets/logo/itu_logo.png)

# Get data from Giga Maps

In [ ]:
pip install delta-sharing pycountry

In [ ]:
import delta_sharing
import pycountry
import geopandas as gpd
import ipywidgets as widgets
from IPython.display import display

In [ ]:
# @title Select country
# Create dropdown: country name shown, ISO3 code stored
country_dropdown = widgets.Dropdown(
    options=sorted([(c.name, c.alpha_3) for c in pycountry.countries]),
    description='Country:',
    value='BRA'
)

display(country_dropdown)

In [ ]:
# Get the key from blob storage
profile_file = "https://zstagigaprodeuw1.blob.core.windows.net/gigainframapkit-public-container/brazil/profile_prd.share"

In [ ]:
# Create a Delta Sharing client and list all tables
client = delta_sharing.SharingClient(profile_file)
client.list_all_tables()

In [ ]:
# Load the table and filter by country
table_url = profile_file + f"#gold.school-master.{country_dropdown.value}"
school_data = delta_sharing.load_as_pandas(table_url)

In [ ]:
# Display the first few rows of the data
school_data.head()

In [ ]:
# Filter the data to include only relevant columns
school_data = school_data[["school_data_source","school_name","latitude","longitude","education_level","num_students","electricity_availability","electricity_type","connectivity_type"]]

In [ ]:
# Convert the DataFrame to a GeoDataFrame
school_data = gpd.GeoDataFrame(school_data, geometry=gpd.points_from_xy(school_data.longitude, school_data.latitude), crs="4326")

In [ ]:
# Save the filtered data to a CSV file
school_data.to_file("school_data.geojson", driver="GeoJSON")